In [1]:
import pandas as pd
import numpy as np

In [2]:
review = pd.read_csv(r'D:\final project\DataBase\Input\01. SkincareReview.csv')
review

,Brand,Product,UserName,SkinCond_Age,Recommend,PostDate,Review,Rating
0,Hada Labo,Perfect 3D Gel,saffanazian,"Oily, 19 - 24",saffanazian recommends this product!,15 hours ago,"udah pake 6 botol , teksturnya unik lebih kaya...",5
1,Hada Labo,Perfect 3D Gel,nadh_,"Normal, 19 - 24",nadh_ recommends this product!,a day ago,Aku pake ini buat night creamku. Ini bener2 bi...,5
2,Hada Labo,Perfect 3D Gel,Ismawf,18 and Under,Ismawf doesn't recommend this product!,2 days ago,Sya cocok sama semua produk hadalabo tapi kecu...,3
3,Hada Labo,Perfect 3D Gel,denishaap,"Normal, 19 - 24",denishaap recommends this product!,3 days ago,alhamdulillah cocok xx dikulitku yang suka rew...,5
4,Hada Labo,Perfect 3D Gel,kifa,"Combination, 30 - 34",kifa recommends this product!,10 Sep 2020,iseng beli nyobain ini dan wowww bekas jerawat...,4
...,...,...,...,...,...,...,...,...
7631,Clinelle,Whiten Up Brightening Night Cream,janejen,"Oily, 18 and Under",janejen recommends this product!,15 Oct 2019,Beli ini gara gara di rekomendadi mba mba guar...,4
7632,Clinelle,Whiten Up Brightening Night Cream,septianalaras,"Combination, 19 - 24",septianalaras recommends this product!,12 Oct 2019,awalnya ragu coba karena harganya di atas budg...,4
7633,Clinelle,Whiten Up Brightening Night Cream,Finatulh,"Normal, 19 - 24",Finatulh recommends this product!,08 Oct 2019,"pake ini barengan sama day creamnya, pake male...",5
7634,Clinelle,Whiten Up Brightening Night Cream,indahsario,"Dry, 19 - 24",NaN,24 Sep 2019,Aku nyoba ini dari sample size tapi langsung j...,4


In [3]:
pip install sastrawi

Note: you may need to restart the kernel to use updated packages.


In [4]:
## Lematisasi
from nltk.stem.wordnet import WordNetLemmatizer #untuk Bahasa Inggris
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory #untuk Bahasa Indonesia
## stopwords
from nltk.corpus import stopwords
## punctuation
import string
import re

#-----------
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
 
punc = set(string.punctuation)
 
#lemmatisasi untuk bahasa Indonesia
factory = StemmerFactory()
stemmer = factory.create_stemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans("","",string.punctuation))
    text = re.sub(r"\d+", "", text)
    text = text.split()
    text = [stemmer.stem(word) for word in text] 
    return ' '.join(text)

In [6]:
review['clean_text'] = review['Review'].apply(clean_text)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
bag_of_word = vectorizer.fit_transform(review['clean_text'])

In [8]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer().fit(bag_of_word)
tfidf = tfidf.transform(bag_of_word)

# modeling

In [9]:
review['positivity'] = review['Rating'].apply(lambda x : 1 if x>3 else -1 if x<3 else 0)

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(tfidf, review['positivity'], test_size = 0.3, random_state = 22)

In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report

In [12]:
rfc = RandomForestClassifier()
rfc.fit(x_train, y_train)
predict = rfc.predict(x_test)
print(classification_report(y_test, predict)) 

              precision    recall  f1-score   support

          -1       0.50      0.01      0.02       284
           0       0.67      0.02      0.04       431
           1       0.69      1.00      0.82      1576

    accuracy                           0.69      2291
   macro avg       0.62      0.34      0.29      2291
weighted avg       0.66      0.69      0.57      2291



In [13]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
predict = logreg.predict(x_test)
print(classification_report(y_test, predict))  

              precision    recall  f1-score   support

          -1       0.60      0.25      0.36       284
           0       0.45      0.19      0.27       431
           1       0.75      0.95      0.84      1576

    accuracy                           0.72      2291
   macro avg       0.60      0.46      0.49      2291
weighted avg       0.68      0.72      0.67      2291



In [14]:
mul_nb = MultinomialNB()
mul_nb.fit(x_train, y_train)
predict = mul_nb.predict(x_test)
print(classification_report(y_test, predict)) 

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       284
           0       0.00      0.00      0.00       431
           1       0.69      1.00      0.82      1576

    accuracy                           0.69      2291
   macro avg       0.23      0.33      0.27      2291
weighted avg       0.47      0.69      0.56      2291



# Hilangin lemmatisasi

In [15]:
stop = list(set(stopwords.words('indonesian'))) 
stop.extend(review['Brand'].apply(lambda x : x.lower()).unique().tolist())
punc = set(string.punctuation)
 
#lemmatisasi untuk bahasa Indonesia
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [28]:
review['Review'][14]

'Produk the best banget jatuh cinta sama ini produk gak tau mau bilang apa lagi tiap pagi sama malem sllu aku pake sblm pake day cream hasilnya sebagus itu tuhaaan ❤️❤️❤️ karena waktu di kaish itu barang endorse tapi ternyata aku jatuh cinta banget'

In [29]:
review['clean_text'][14]

'produk the best banget jatuh cinta produk gak tau bilang pagi malem sllu pake sblm pake day cream hasilnya sebagus tuhaaan ❤️❤️❤️ kaish barang endorse jatuh cinta banget'

In [16]:
def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans("","",string.punctuation))
    text = re.sub(r"\d+", "", text)
    text = text.split()
    text = [word for word in text if word not in stop] 
    return ' '.join(text)

In [17]:
review['clean_text'] = review['Review'].apply(clean_text)

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
bag_of_word = vectorizer.fit_transform(review['clean_text'])

In [20]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer().fit(bag_of_word)
tfidf = tfidf.transform(bag_of_word)

In [21]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(tfidf, review['positivity'], test_size = 0.3, random_state = 22)

In [22]:
rfc = RandomForestClassifier()
rfc.fit(x_train, y_train)
predict = rfc.predict(x_test)
print(classification_report(y_test, predict)) 

              precision    recall  f1-score   support

          -1       0.60      0.01      0.02       284
           0       0.64      0.02      0.03       431
           1       0.69      1.00      0.82      1576

    accuracy                           0.69      2291
   macro avg       0.64      0.34      0.29      2291
weighted avg       0.67      0.69      0.57      2291



In [23]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
predict = logreg.predict(x_test)
print(classification_report(y_test, predict))  

              precision    recall  f1-score   support

          -1       0.62      0.21      0.31       284
           0       0.47      0.17      0.25       431
           1       0.75      0.96      0.84      1576

    accuracy                           0.72      2291
   macro avg       0.61      0.45      0.47      2291
weighted avg       0.68      0.72      0.66      2291



In [24]:
mul_nb = MultinomialNB()
mul_nb.fit(x_train, y_train)
predict = mul_nb.predict(x_test)
print(classification_report(y_test, predict)) 

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       284
           0       0.00      0.00      0.00       431
           1       0.69      1.00      0.81      1576

    accuracy                           0.69      2291
   macro avg       0.23      0.33      0.27      2291
weighted avg       0.47      0.69      0.56      2291

